Computational appendix of [arXiv:2212.09765](https://www.arxiv.org/abs/2212.09765)
---

In this notebook we perform a step-by-step implementation of the hybrid classical-nonsignaling inflation of the three-branch scenario presented in [arXiv:2212.09765](https://www.arxiv.org/abs/2212.09765) and obtain the full network nonlocality witnesses presented there, namely Equation (4).

Authors: Alejandro Pozas-Kerstjens

Requires: [mosek](https://www.mosek.com/) >= 9.1.13 & <= 9.2.28, [numpy](https://numpy.org/), [picos](https://picos-api.gitlab.io/picos/) <= 2.2.55, [qutip](http://qutip.org/), [sympy](https://www.sympy.org/)

Last updated: 21 Dec, 2022

## Introduction

In this notebook we follow the first part in the Materials and Methods of the manuscript in order to implement the linear program arising from application of the inflation technique to detect whether a four-partite distribution admits a realisation in the three-branch star network when one of the sources is assumed to be classical and the other ones distribute general no-signaling resources. We will implement, step by step, the linear program defined there through direct implementation of Equations (6-10), solving the problem, extracting the associated certificate of infeasibility, and deriving from it the full network nonlocality witness in Equation (4) in the manuscript.

We begin by importing the necessary libraries and by defining several objects that will be useful throughout the notebook.

In [1]:
import numpy as np
import qutip as qt
import sympy as sp
import picos as pic

from sympy import Symbol as S

In [2]:
oA=2; mA=2; oB=2;

l = r'\langle{}'; r = r'\rangle{}';    # For pretty visuals

Now we define the family of probability distributions that we consider. These are generated by measuring a (potentially noisy) maximally entangled state between each of the branch parties and the central party, the latter performing a two-outcome measurement described by the projectors $|\text{GHZ}\rangle\langle\text{GHZ}|$ and $1-|\text{GHZ}\rangle\langle\text{GHZ}|$, and the formers being able to perform the same two binary-outcome measurements defined on the X-Z plane, characterized by angles $\theta_0$ and $\theta_1$.

In [3]:
def prob(v, θ0, θ1):
    ϕplus = qt.ket2dm(qt.bell_state('00'))
    ϕplus.dims = [[4], [4]]
    ghzplus = qt.ket2dm((qt.basis([2, 2, 2], [0, 0, 0])
                         + qt.basis([2, 2, 2], [1, 1, 1]))
                        / np.sqrt(2))
    ghzminus = qt.ket2dm((qt.basis([2, 2, 2], [0, 0, 0])
                          - qt.basis([2, 2, 2], [1, 1, 1]))
                         / np.sqrt(2))
    id2 = qt.qeye(2); id4 = qt.qeye(4);

    ρ = qt.tensor([v*ϕplus+(1-v)*id4/4,
                   v*ϕplus+(1-v)*id4/4,
                   v*ϕplus+(1-v)*id4/4])
    ρ.dims = [[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]]
    ρ = ρ.permute([0,2,4,1,3,5])

    A = [np.sin(θ0)*qt.sigmax() + np.cos(θ0)*qt.sigmaz(),
         np.sin(θ1)*qt.sigmax() + np.cos(θ1)*qt.sigmaz()]
    B = [ghzplus, qt.qeye([2, 2, 2]) - ghzplus]

    p = np.zeros((oA,oA,oA,oB,mA,mA,mA))
    for a1,a2,a3,b,x1,x2,x3 in np.ndindex(oA,oA,oA,oB,mA,mA,mA):
        p[a1,a2,a3,b,x1,x2,x3] = np.real(ρ.overlap(
            qt.tensor([id2+(-1)**a1*A[x1], id2+(-1)**a2*A[x2],
                       id2+(-1)**a3*A[x3], B[b]]))/8)
    return p

Next, we define the function that will set up the linear program corresponding to compatibility with the inflation in Fig. 4 in the manuscript, where the information sent from the source connecting the central party with $A^{(1)}$ is assumed to be encoded in a classical system, and thus is copied and sent to multiple copies of $A^{(1)}$. We will do so using the library [PICOS](https://picos-api.gitlab.io/picos/), which will allow us to define the probability distribution $p_{\text{inf}}$ subject to the normalization, no-signaling and marginalization constraints, as well as to symmetry constraints derived from the fact that there exist multiple copies of the party $A^{(1)}$. Within PICOS we will also extract the certificate of infeasibility in case the linear program does not admit any solution, and later transform this into a Bell-like inequality.

In [4]:
def set_problem(prob_dist, solve=False):
    oA = prob_dist.shape[0]
    oB = prob_dist.shape[3]
    mA = prob_dist.shape[4]
    probabilities = pic.RealVariable('pinf',
                                     shape=oA*oA*oA*oA*oA*oB*mA*mA*mA*mA*mA,
                                     lower=0) # Directly impose positivity (Eq. 6) here
    pinf = np.reshape(probabilities,
                      (oA,oA,oA,oA,oA,oB,mA,mA,mA,mA,mA))

    # Normalisation (Eq. 7)
    norm_const = [probsum == 1
                  for probsum in np.sum(pinf, (0, 1, 2, 3, 4, 5)
                                       ).flatten().tolist()]

    # No-signaling for party A11 (Eq. 8)
    nsA_const = [[] for _ in range(mA-1)]
    nsA_0_terms = np.sum(pinf[:,:,:,:,:,:,0,:,:,:,:], 0)
    for x in range(1, mA):
        nsA_x_terms = np.sum(pinf[:,:,:,:,:,:,x,:,:,:,:], 0)
        nsA_const[x-1] = [term == 0 for term in
                          (nsA_0_terms - nsA_x_terms).flatten().tolist()]
    nsA1_const = sum(nsA_const, [])
    # We do not need to impose no-signaling on A12 and A13
    # because they will be enforced when setting the 
    # inflation symmetries

    # No-signaling for party A2 (Eq. 8)
    nsA_const = [[] for _ in range(mA-1)]
    nsA_0_terms = np.sum(pinf[:,:,:,:,:,:,:,:,:,0,:], 3)
    for x in range(1, mA):
        nsA_x_terms = np.sum(pinf[:,:,:,:,:,:,:,:,:,x,:], 3)
        nsA_const[x-1] = [term == 0 for term in
                          (nsA_0_terms - nsA_x_terms).flatten().tolist()]
    nsA2_const = sum(nsA_const, [])
    
    # No-signaling for party A3 (Eq. 8)
    nsA_const = [[] for _ in range(mA-1)]
    nsA_0_terms = np.sum(pinf[:,:,:,:,:,:,:,:,:,:,0], 4)
    for x in range(1, mA):
        nsA_x_terms = np.sum(pinf[:,:,:,:,:,:,:,:,:,:,x], 4)
        nsA_const[x-1] = [term == 0 for term in
                          (nsA_0_terms - nsA_x_terms).flatten().tolist()]
    nsA3_const = sum(nsA_const, [])

    # Inflation (Eq. 9). A11 <-> A12
    inf_const = [term == 0 for term in
                 (pinf-np.transpose(pinf, [1,0,2,3,4,5,7,6,8,9,10])
                  ).flatten().tolist()]
    # Inflation (Eq. 9). A11 <-> A13
    inf_const += [term == 0 for term in
                  (pinf-np.transpose(pinf, [2,1,0,3,4,5,8,7,6,9,10])
                   ).flatten().tolist()]
    # Inflation (Eq. 9). A12 <-> A13
    inf_const += [term == 0 for term in
                  (pinf-np.transpose(pinf, [0,2,1,3,4,5,6,8,7,9,10])
                   ).flatten().tolist()]
    # Inflation (Eq. 9). A11 -> A12 -> A13 -> A11
    inf_const += [term == 0 for term in
                  (pinf-np.transpose(pinf, [1,2,0,3,4,5,7,8,6,9,10])
                   ).flatten().tolist()]
    # Inflation (Eq. 9). A11 -> A13 -> A12 -> A11
    inf_const += [term == 0 for term in
                  (pinf-np.transpose(pinf, [2,0,1,3,4,5,8,6,7,9,10])
                   ).flatten().tolist()]

    # Matching original probabilities (Eq. 10)
    marginal_const = [term == 0 for term in
                      (np.sum(pinf[:,:,:,:,:,:,:,0,0,:,:], (1, 2))
                       -prob_dist).flatten().tolist()]
    
    # Set up the problem
    problem = pic.Problem()
    for const in (norm_const + nsA1_const + nsA2_const
                  + nsA3_const + inf_const + marginal_const):
        problem.add_constraint(const)

    # Solve if requested
    if solve:
        try:
            problem.solve(solver='mosek')
            print('Optimal solution found')
            return [np.reshape(probabilities.value,
                               (oA,oA,oA,oA,oA,oB,mA,mA,mA,mA,mA)),
                    [None, None]]
        except pic.SolutionFailure:
            print('The problem is infeasible. Getting certificate...')
            problem.solve(solver='mosek', primals=False)            
            print('Primal or dual infeasibility certificate found')
            ynorm     = np.array([const.dual for const in norm_const])
            ymarginal = np.array([const.dual for const in marginal_const])
            return [None, [ynorm, ymarginal]]
    else:
        return problem

In [5]:
[pinf, [ynorm, ymarginal]] = set_problem(prob(0.882, -1.865, -0.4146),
                                         solve=True)

The problem is infeasible. Getting certificate...
Primal or dual infeasibility certificate found


The certificate of infeasibility is a vector $\mathbf{y}$ that satisfies $\mathbf{y}\cdot A = \mathbf{0}$ and $\mathbf{y}\cdot\mathbf{b} > 0$, which signals the impossibility of finding a vector $\mathbf{x}$ that satisfies $A\cdot\mathbf{x} \geq \mathbf{b}$. Importantly, since in the problem we are dealing with the matrix $A$ is constant regardless the distribution that we want to study, the quantity $\mathbf{y}\cdot\mathbf{b} > 0$ can be understood as a Bell-like inequality that applies not only to the distribution under scrutiny but also to arbitrary ones. In the following, we write the inequality in terms of arbitrary probabilities, to read it as a Bell-like inequality.

In [6]:
# Build symbols for the probabilities
symp = np.zeros((oA,oA,oA,oB,mA,mA,mA), dtype=object)
for a1,a2,a3,b,x1,x2,x3 in np.ndindex(oA,oA,oA,oB,mA,mA,mA):
    symp[a1,a2,a3,b,x1,x2,x3] = sp.symbols(
        f"p({a1}{a2}{a3}{b}|{x1}{x2}{x3})")

# Function to write probabilities in terms of expectation values
def prob_to_expect(symbol):
    a1, a2, a3, b, _, x1, x2, x3 = str(symbol)[2:-1]
    a1 = int(a1); a2 = int(a2); a3 = int(a3); b = int(b)
    
    expec = 1/16 * (
        1
        + (-1)**b * S(l + "B" + r)
        + (-1)**a1 * S(l + "A^{(1)}_" + x1 + r)
        + (-1)**a2 * S(l + "A^{(2)}_" + x2 + r)
        + (-1)**a3 * S(l + "A^{(3)}_" + x3 + r)
        + (-1)**(a1 + b) * S(l + "A^{(1)}_" + x1 + "B" + r)
        + (-1)**(a2 + b) * S(l + "A^{(2)}_" + x2 + "B" + r)
        + (-1)**(a3 + b) * S(l + "A^{(3)}_" + x3 + "B" + r)
        + (-1)**(a1 + a2) * S(l + "A^{(1)}_" + x1 + "A^{(2)}_" + x2 + r)
        + (-1)**(a1 + a3) * S(l + "A^{(1)}_" + x1 + "A^{(3)}_" + x3 + r)
        + (-1)**(a2 + a3) * S(l + "A^{(2)}_" + x2 + "A^{(3)}_" + x3 + r)
        + (-1)**(a1 + a2 + b) * S(l + "A^{(1)}_" + x1 + "A^{(2)}_" + x2 + "B" + r)
        + (-1)**(a1 + a3 + b) * S(l + "A^{(1)}_" + x1 + "A^{(3)}_" + x3 + "B" + r)
        + (-1)**(a2 + a3 + b) * S(l + "A^{(2)}_" + x2 + "A^{(3)}_" + x3 + "B" + r)
        + (-1)**(a1 + a2 + a3) * S(l + "A^{(1)}_" + x1 + "A^{(2)}_" + x2 + "A^{(3)}_" + x3 + r)
        + (-1)**(a1 + a2 + a3 + b) * S(l + "A^{(1)}_" + x1 + "A^{(2)}_" + x2 + "A^{(3)}_" + x3 + "B" + r)
    )
    return expec

In [7]:
# Extract certificate in terms of probabilities
I1 = ymarginal @ symp.flatten() + sum(ynorm)

# Write in terms of correlators
16*I1.subs({prob: prob_to_expect(prob) for prob in I1.free_symbols})

-1.0*\langle{}A^{(1)}_0A^{(2)}_0A^{(3)}_0B\rangle{} - 1.0*\langle{}A^{(1)}_0A^{(2)}_0A^{(3)}_0\rangle{} - 1.0*\langle{}A^{(1)}_0A^{(2)}_0A^{(3)}_1B\rangle{} - 1.0*\langle{}A^{(1)}_0A^{(2)}_0A^{(3)}_1\rangle{} - 1.0*\langle{}A^{(1)}_0A^{(3)}_0B\rangle{} - 1.0*\langle{}A^{(1)}_0A^{(3)}_0\rangle{} - 1.0*\langle{}A^{(1)}_0A^{(3)}_1B\rangle{} - 1.0*\langle{}A^{(1)}_0A^{(3)}_1\rangle{} - 1.0*\langle{}A^{(1)}_1A^{(2)}_0A^{(3)}_0B\rangle{} - 1.0*\langle{}A^{(1)}_1A^{(2)}_0A^{(3)}_0\rangle{} + \langle{}A^{(1)}_1A^{(2)}_0A^{(3)}_1B\rangle{} + \langle{}A^{(1)}_1A^{(2)}_0A^{(3)}_1\rangle{} - 1.0*\langle{}A^{(1)}_1A^{(3)}_0B\rangle{} - 1.0*\langle{}A^{(1)}_1A^{(3)}_0\rangle{} + \langle{}A^{(1)}_1A^{(3)}_1B\rangle{} + \langle{}A^{(1)}_1A^{(3)}_1\rangle{} - 2.0*\langle{}A^{(2)}_0B\rangle{} - 2.0*\langle{}A^{(2)}_0\rangle{} - 2.0*\langle{}B\rangle{} - 2.0

This quantity, $\mathcal{I}_1$, is negative for all distributions that can be generated in the three-branch star network when the source connecting $A^{(1)}$ to the central party is classical, and thus obtaining a positive value signals that the distribution that achieves this value cannot be produced using a classical source there. Because the star network considered is symmetric under cyclic permutations of the branch parties, $\mathcal{I}_2$ and $\mathcal{I}_3$, corresponding to the situations where the classical sources are the one connecting, respectively, $A^{(2)}$ and $A^{(3)}$ with $B$, can be obtained by performing the permutations $A^{(1)}\rightarrow A^{(2)}\rightarrow A^{(3)} \rightarrow A^{(1)}$ and $A^{(1)}\rightarrow A^{(3)}\rightarrow A^{(2)} \rightarrow A^{(1)}$. A simultaneous violation of $\mathcal{I}_1\leq 0$, $\mathcal{I}_2\leq 0$ and $\mathcal{I}_3\leq 0$ by a probability distribution implies that the distribution is fully network nonlocal.